In [81]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np
import pandas as pd
import pickle 

### helper functions

In [82]:
def helper_regex(text):
    match = re.search(rf"{'Full'}\s+(\w+)", text)
    if match:
        return match.group(1)
    return None

def cast_func(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities

    for i in range(4000, 20000): 
        data_intensities.append(0) 
        data_masses.append(i/10)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 4000 + 1)/10 > scan_masses[j] > (i + 4000)/10: 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]

    # for i in range(0,16000):
    #     if (precursor -10) < data_masses[i] < (precursor +10): data_intensities[i] = 0
        
    return(list(data_intensities))

def cast_func_1amu(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 
 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
  
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities
   
    # for k in range(0, len(scan_intensities)):
    #     if scan_intensities[k] > (5/100)*max(scan_intensities): 
    #         scan_intensities1.append(scan_intensities[k])
    #         scan_masses1.append(scan_masses[k])


    for i in range(400, 2000): 
        data_intensities.append(0) 
        data_masses.append(i)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 400 + 1) > scan_masses[j] > (i + 400): 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]

    # for i in range(0,16000):
    #     if (precursor -10) < data_masses[i] < (precursor +10): data_intensities[i] = 0
        
    return(list(data_intensities))



### select a raw file

In [92]:
raw = RawFile("C:/20231215_msz1718_MAbPac-06_Neo_ACN1_TreatmentA_biorep01_techrep01.raw")
# raw = RawFile("C:/20231221_msz1718_MAbPac-02_Neo_ExtraValveT_ACN30S1_TreatmentA_biorep01_techrep01.raw")

### generate a cast spectra database including scan index, retention time, precursor m/z values, and TIC

In [94]:
cast_spectra = []
mz_value = []
tic = []
scan_number = []
retention_time = []

for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        cast_spectra.append(cast_func_1amu(i))
        mz_value.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))
        tic.append(raw_scan.scan_statistics.tic)
        scan_number.append(raw_scan.scan_statistics.scan_number)
        retention_time.append(raw.get_retention_time_from_scan_number(raw_scan.scan_statistics.scan_number))
        print("scan", i, "is complete")

scan_dict = {'scan': scan_number, 'retntion time': retention_time, 'm/z': mz_value, 'TIC': tic, 'cast spectra': cast_spectra} 

with open('scan_dict_run1', 'wb') as f: 
     pickle.dump(scan_dict, f) 

print("completed")

scan 3 is complete
scan 116 is complete
scan 229 is complete
scan 295 is complete
scan 366 is complete
scan 407 is complete
scan 424 is complete
scan 466 is complete
scan 467 is complete
scan 469 is complete
scan 470 is complete
scan 472 is complete
scan 473 is complete
scan 475 is complete
scan 476 is complete
scan 478 is complete
scan 479 is complete
scan 481 is complete
scan 482 is complete
scan 484 is complete
scan 485 is complete
scan 487 is complete
scan 488 is complete
scan 490 is complete
scan 491 is complete
scan 493 is complete
scan 494 is complete
scan 496 is complete
scan 497 is complete
scan 499 is complete
scan 500 is complete
scan 502 is complete
scan 503 is complete
scan 505 is complete
scan 506 is complete
scan 508 is complete
scan 509 is complete
scan 511 is complete
scan 512 is complete
scan 513 is complete
scan 515 is complete
scan 516 is complete
scan 518 is complete
scan 519 is complete
scan 521 is complete
scan 522 is complete
scan 524 is complete
scan 525 is com

In [ ]:
pd.DataFrame(scan_dict)

In [95]:
with open('scan_dict_run1', 'rb') as f: 
     scan_dict_run1 = pickle.load(f) 

with open('scan_dict_run2', 'rb') as f: 
     scan_dict_run2 = pickle.load(f) 

### search a complete run against a database

In [96]:
pd.DataFrame(scan_dict_run1)

,scan,retntion time,m/z,TIC,cast spectra
0,3,0.012763,532.8568,44524.265625,"[0, 142.436279296875, 97.23536682128906, 0, 0,..."
1,116,1.018793,532.6563,44763.757812,"[0, 0, 96.82411193847656, 0, 0, 0, 0, 0, 0, 0,..."
2,229,2.024847,532.6566,46074.152344,"[0, 0, 0, 0, 91.3983383178711, 0, 0, 0, 0, 0, ..."
3,295,2.616064,533.2579,43067.757812,"[263.31329345703125, 94.55951690673828, 0, 85...."
4,366,3.251314,665.5680,26676.625000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 85.980964660..."
...,...,...,...,...,...
6201,9552,118.289241,1254.4952,129871.250000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6202,9553,118.311373,1220.9719,14413.310547,"[0, 0, 0, 0, 0, 0, 0, 0, 97.58184051513672, 0,..."
6203,9555,118.337616,1081.2610,10096.425781,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6204,9556,118.359320,1331.7272,19646.671875,"[0, 0, 0, 0, 0, 75.4213638305664, 0, 0, 0, 0, ..."


In [ ]:
database_1 = scan_dict_run1["cast spectra"]
database_2 = scan_dict_run2["cast spectra"]

mz_value_1 = scan_dict_run1["m/z"]
mz_value_2 = scan_dict_run2["m/z"]

scan_index_1 = scan_dict_run1["scan"]
scan_index_2 = scan_dict_run2["scan"]

similarity = []
match_scan = []

for i in range(0, len(database_2)):
    similarity.append(0)
for i in range(0, len(database_2)):
    match_scan.append(0)


for i in range(0, len(database_2)):
    for j in range(0, len(database_1)):
        if mz_value_2[i] - 1 < mz_value_1[j] < mz_value_2[i] + 1:
            X = [database_2[i], database_1[j]]
            if round(cosine_similarity(X)[1][0], 2) > similarity[i]: 
                similarity[i] = round(cosine_similarity(X)[1][0], 2)
                match_scan[i] = scan_index_1[j]
    print("the max similarity for index", i, "is", similarity[i], "found in index ", match_scan[i])

In [102]:
scan_dict_run2['similarity'] = similarity
scan_dict_run2['match_scan'] = match_scan
del scan_dict_run2['cast spectra']

In [105]:
df = pd.DataFrame(scan_dict_run2)
df.to_csv('result.csv')


### run a single spectra against a database

In [ ]:
ref_scan = cast_func_1amu(10244)
scan_id = []
scan_sim = []
precursor = []

for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        X = [ref_scan, cast_func_1amu(i)]
        result = round(cosine_similarity(X)[1][0], 2)       
        precursor.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))
        scan_id.append(i)
        scan_sim.append(result)
        print(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]), "in scan", i, "is", result)

np.savetxt("outputall.csv", np.transpose(np.vstack((np.array(scan_id), np.array(precursor), np.array(scan_sim)))) , delimiter=",")